In [1]:
#define hostname
HOSTNAME = '192.168.1.100'

In [2]:
import os
HOSTNAME = os.environ['REDPITAYA_HOSTNAME']

In [ ]:
import pyrpl
p = pyrpl.Pyrpl(config="trig_maxhold_test",  # do not use a config file 
                hostname=HOSTNAME)
rp = p.redpitaya  # shortcut for the the redpitaya handler

In [ ]:
import numpy as np

def make_custom_waveform():
    """ 
    This waveform imitates very short pulses of varying amplitude.
    
    See scope traces of this waveform rather than trying to infer 
    its shape from code.
    """
    waveform = np.zeros(2**14)
    for i in range(16):
        for j in range(100):
            waveform[i*1000+j] = i*0.01+j*0.008
        jmax = j
        vmax = waveform[i*1000+j]
        for j in range(100):
            waveform[i*1000+j+jmax] = vmax-j*0.007
    return waveform

# setup asgs with custom waveform
for asg in [rp.asg0, 
            #rp.asg1
           ]:
    # general asg setup
    asg.setup(waveform='ramp', 
              amplitude=1.0, 
              offset=0, 
              frequency=1e5, 
              output_direct='off',
              trigger_source='immediately')
    # hack to load the custom waveform into asg
    asg.data = make_custom_waveform()
    
# use trig module for max-mold mechanism
trig = rp.trig
trig.setup(input='asg0',  # input signal
           output_direct='off',  # whether the output should be summed to a voltage output out1 or out2
           output_signal='max',  # sets up max-hold modus, min-hold is also possible with 'min'
           trigger_source='pos_edge',  # trigger modus, i.e. positive slope, negative slope, etc.
           threshold=0.1,  # voltage threshold above (pos_edge) or below (neg_edge) a max-hold acquisition is started
           hysteresis=0.005,  # hysteresis for the trigger to detect an edge
           auto_rearm_delay=0.01/asg.frequency,  # the delay in seconds afte trigger event during which max-/min is measured
           auto_rearm=True,  # if False, only a single max/min is computed and output until next reset of the module
          )

# manually check on the scope
rp.scope.setup(input1='asg0',
               input2='trig',
               duration=5.0/asg.frequency,
               average=False,
               trigger_source='asg0',
               rolling_mode=False,
               running_state='running_continuous'
               )

# have a look at the scope gui, this should be self-explanatory

In [ ]:
# unrelated, but often-demanded feature on taking into account darknoise from lockbox inputs
p.lockbox.inputs.port1.get_analog_offset?